Próximos passos:
- Funis;

# Conectando com o Drive

In [1]:
from google.colab import drive, auth
from google.auth import default

drive.mount('/content/drive/')
auth.authenticate_user()
creds, _ = default()

Mounted at /content/drive/


# Importando Bibliotecas

In [2]:
import os
import re
import gspread
import numpy as np
import pandas as pd
import datetime as dt
from dateutil import relativedelta
from openpyxl.utils.cell import get_column_letter

gc = gspread.authorize(creds)

# Executando a ferramenta

In [ ]:
'''
Ordem das métricas a serem exibiudas no relatório
'''
focus_columns = [
    'Plataforma', 'Data', 'Período', 'Investimento', 'Impressões',
    'Cliques', 'CPC', 'CPM', 'CTR', 'Leads',
    'CPL', 'Sessões', 'Custo por Sessão', 'Taxa de Conversão de Compras', 'Compras',
    'Compras Mídia', 'Compras Total', 'CPA', 'CPA Mídia', 'CPA Total',
    'Receita', 'Receita Mídia', 'Receita Total', 'Ticket Médio', 'Ticket Médio Mídia',
    'Ticket Médio Total', 'ROAS', 'ROAS Mídia', 'ROAS Total'
]

'''
Essa função recebe o editor de google sheets e o nome de uma aba específica, realiza a leitura dos dados
nela presentes e retorna um dataframe com esses dados organizados.
'''
def read_worksheet(sh, sheet_name):
  df = pd.DataFrame(sh.worksheet(sheet_name).get_all_values())
  df.columns = df.iloc[0].tolist()
  df = df.drop([0]).reset_index(drop=True)

  return df

'''
Essa função recebe a URL do documento de Configuração que será usado durante todo o processamento
'''
def open_config(config_url):
  config_sh = gc.open_by_url(config_url)
  print(f'Documento de Configuração aberto')

  translation_df = read_worksheet(config_sh, 'Dicionário')
  config_df = read_worksheet(config_sh, 'Relatórios')

  return translation_df, config_df


'''
Essa função recebe um parâmetro obrigatório: a url do arquivo google sheets da base de dados.
Os demais dois parâmetros são facultativos. Caso já exista um report criado, fornecer a url do mesmo.
Caso deva-se criar um novo report, fornecer o nome do novo arquivo a ser criado.
Essa função retorna o leitor/editor dos arquivos e a lista de diferentes bases de dados disponíveis.
'''
def open_reports(db_url, report_url, client_name):
  db_sh = gc.open_by_url(db_url)
  print(f'DataBase "{db_sh.title}" aberta')

  if report_url:
    report_sh = gc.open_by_url(report_url)
    print(f'Report "{report_sh.title}" aberto')
  else:
    new_report_name = f"{client_name} - Report"
    report_sh = gc.create(new_report_name)
    print(f'Report "{new_report_name}" criado')

  if db_sh.title.split(' - ')[0] != report_sh.title.split(' - ')[0]:
    print('DataBase e Report diferentes')
    print('Por favor, cheque os links colocados no documento de Configuração')
    print('-----------------------------------------------------------------')
    return None

  db_names_list = [str(db).split("'")[1] for db in db_sh.worksheets() if 'bd_' in str(db).split("'")[1]]

  return db_sh, report_sh, db_names_list

'''
Essa função recebe uma data e a converte para o formato string AAAA-MM-DD.
Caso o parâmetro timestamp seja True, também mostra a hora para a data fornecida.
'''
def date_to_string(d, timestamp=False):
  if timestamp:
    d = d - relativedelta.relativedelta(hours=3)
    return d.strftime('%Y-%m-%d %H:%M:%S')

  return d.strftime('%Y-%m-%d')

'''
Essa função recebe um dicionário de configuração, o qual é preenchido na seção "Execução do Script".
A partir das datas fornecidas, calcula-se as datas referentes à semana, mês e ano anteriores.
Para controle do ano corrente, também calcula o primeiro dia do ano e o dia anterior da geração do report.
Após os cálculos, um dicionário com todas as datas é retornado.
'''
def get_dates_dict(**kwargs):
  init_date = dt.datetime.strptime(kwargs.get('init_date'), "%Y-%m-%d")
  end_date = dt.datetime.strptime(kwargs.get('end_date'), "%Y-%m-%d")

  custom_init_date = kwargs.get('custom_init_date')
  custom_end_date = kwargs.get('custom_end_date')

  todays_date = dt.date.today()
  init_current_year = todays_date.replace(month=1, day=1)
  yesterdays_date = todays_date - relativedelta.relativedelta(days=1)

  dates_dict = {
      'init_current_year': init_current_year,
      'yesterdays_date': yesterdays_date,
      'init_date': init_date,
      'end_date': end_date,
      'wow_init_date': init_date - relativedelta.relativedelta(days=7),
      'wow_end_date': end_date - relativedelta.relativedelta(days=7),
      'mom_init_date': init_date - relativedelta.relativedelta(months=1),
      'mom_end_date': end_date - relativedelta.relativedelta(months=1),
      'yoy_init_date': init_date - relativedelta.relativedelta(years=1),
      'yoy_end_date': end_date - relativedelta.relativedelta(years=1)
  }

  if custom_init_date and custom_end_date:
    dates_dict['custom_init_date'] = dt.datetime.strptime(custom_init_date, "%Y-%m-%d")
    dates_dict['custom_end_date'] = dt.datetime.strptime(custom_end_date, "%Y-%m-%d")

  return dates_dict

'''
Essa função recebe um dicionário de configuração, o qual é preenchido na seção "Execução do Script".
A partir das datas fornecidas, executa a função que calcula as datas e retorna um dicionário com os respectivos períodos.
Exemplo "2024-01-01 a 2024-01-31"
'''
def determine_periods(**kwargs):
  dates_dict = get_dates_dict(**kwargs)

  periods_dict = {
      'current_period': f"{date_to_string(dates_dict['init_date'])} a {date_to_string(dates_dict['end_date'])}",
      'wow_period': f"{date_to_string(dates_dict['wow_init_date'])} a {date_to_string(dates_dict['wow_end_date'])}",
      'mom_period': f"{date_to_string(dates_dict['mom_init_date'])} a {date_to_string(dates_dict['mom_end_date'])}",
      'yoy_period': f"{date_to_string(dates_dict['yoy_init_date'])} a {date_to_string(dates_dict['yoy_end_date'])}",
  }

  if dates_dict.get('custom_init_date') and dates_dict.get('custom_end_date'):
    periods_dict['custom_period'] = f"{date_to_string(dates_dict['custom_init_date'])} a {date_to_string(dates_dict['custom_end_date'])}"

  return periods_dict

'''
Essa função recebe o editor do report e um dicionário de configuração, o qual é preenchido na seção "Execução do Script".
A partir das datas fornecidas, executa a função que calcula as datas e preenche a sheet de "Configuração" com as datas fornecidas,
bem como indica qual a data e hora da criação/atualização do report
'''
def create_config_sheet(report_sh, **kwargs):
  config_values = []

  dates_dict = get_dates_dict(**kwargs)

  df = pd.DataFrame(columns=['Labels', 'Dates'])

  df['Labels'] = ['Report Creation Date', 'Init Date', 'End Date', 'WoW Init Date', 'WoW End Date', 'MoM Init Date', 'MoM End Date', \
                  'YoY Init Date', 'YoY End Date']

  df['Dates'] = [date_to_string(dt.datetime.today(), True), date_to_string(dates_dict['init_date']), date_to_string(dates_dict['end_date']), \
                 date_to_string(dates_dict['wow_init_date']), date_to_string(dates_dict['wow_end_date']), \
                 date_to_string(dates_dict['mom_init_date']), date_to_string(dates_dict['mom_end_date']), \
                 date_to_string(dates_dict['yoy_init_date']), date_to_string(dates_dict['yoy_end_date'])]

  if dates_dict.get('custom_init_date') and dates_dict.get('custom_end_date'):
    df = df._append([{'Labels':'Custom Init Date','Dates':date_to_string(dates_dict['custom_init_date'])},
     {'Labels':'Custom End Date','Dates':date_to_string(dates_dict['custom_end_date'])}], ignore_index=True)

  config_values.append(['Configuração'])
  config_values.extend(df.values.tolist())
  config_values.append([""])

  update_reports_sheet(report_sh, 'Configuração', config_values)

'''
Essa função recebe o dataframe preenchido com as métricas brutas de cada período presentes no banco de dados original.
Caso um dos períodos WoW, MoM, YoY ou Custom não esteja presente no banco de dados, a coluna referente a esse periódo será removida.
'''
def drop_zero_cols(df):
  for col in df.columns.tolist():
    if (df[col] == 0).all():
      df = df.drop([col], axis=1)

  return df

'''
Essa função recebe o dataframe preenchido com as métricas brutas e manipuladas e calcula
 a variação dessas métricas para cada um dos períodos.
'''
def calculate_periods_variation(df, current_year=None, **periods_dict):
  df.loc['Delta WoW'] = (df.loc[periods_dict.get('current_period')]-df.loc[periods_dict.get('wow_period')])/df.loc[periods_dict.get('wow_period')]
  df.loc['Delta MoM'] = (df.loc[periods_dict.get('current_period')]-df.loc[periods_dict.get('mom_period')])/df.loc[periods_dict.get('mom_period')]
  df.loc['Delta YoY'] = (df.loc[periods_dict.get('current_period')]-df.loc[periods_dict.get('yoy_period')])/df.loc[periods_dict.get('yoy_period')]

  rows = [
      current_year, periods_dict.get('current_period'), periods_dict.get('wow_period'), 'Delta WoW', periods_dict.get('mom_period'),
      'Delta MoM', periods_dict.get('yoy_period'), 'Delta YoY'
      ]

  if periods_dict.get('custom_period'):
    df.loc['Delta Custom'] = (df.loc[periods_dict.get('current_period')]-df.loc[periods_dict.get('custom_period')])/df.loc[periods_dict.get('custom_period')]
    rows.append('Delta Custom')

  df = df.reindex(index = rows)

  return df

'''
Essa função recebe o dataframe preenchido com as métricas brutas de cada período presentes no banco de dados original.
A partir desses dados, calcula as demais métricas manipulando as originais.
Também é responsável por calcular a variação dessas métricas para cada um dos períodos.
'''
def calculate_metrics(df, plataform_name, total_flag=False, current_year=None, **periods_dict):
  cols = df.columns.tolist()

  if plataform_name == 'metaads':
    if set(['Investimento', 'Impressões']) <= set(cols):
      df['CPM'] = df['Investimento']/(df['Impressões']/1000) if not total_flag else df['Investimento']/(df.loc['Total', 'Impressões']/1000)
  else:
    if set(['Investimento', 'Cliques']) <= set(cols):
      df['CPC'] = df['Investimento']/df['Cliques'] if not total_flag else df['Investimento']/df.loc['Total', 'Cliques']

  if set(['Cliques', 'Impressões']) <= set(cols):
    df['CTR'] = df['Cliques']/df['Impressões'] if not total_flag else df['Cliques']/df.loc['Total', 'Impressões']

  if set(['Investimento', 'Compras']) <= set(cols):
    df['CPA'] = df['Investimento']/df['Compras'] if not total_flag else df['Investimento']/df.loc['Total', 'Compras']

  if set(['Investimento', 'Leads']) <= set(cols):
    df['CPL'] = df['Investimento']/df['Leads'] if not total_flag else df['Investimento']/df.loc['Total', 'Leads']

  if set(['Investimento', 'Sessões']) <= set(cols):
    df['Custo por Sessão'] = df['Investimento']/df['Sessões'] if not total_flag else df['Investimento']/df.loc['Total', 'Sessões']

  if set(['Cliques', 'Compras']) <= set(cols):
    df['Taxa de Conversão de Compras'] = df['Compras']/df['Cliques'] if not total_flag else df['Compras']/df.loc['Total', 'Cliques']

  if set(['Receita', 'Compras']) <= set(cols):
    df['Ticket Médio'] = df['Receita']/df['Compras'] if not total_flag else df['Receita']/df.loc['Total', 'Compras']

  if set(['Receita Total', 'Compras Total']) <= set(cols):
    df['Ticket Médio Total'] = df['Receita Total']/df['Compras Total'] if not total_flag else df['Receita Total']/df.loc['Total', 'Compras Total']

  if set(['Receita Mídia', 'Compras Mídia']) <= set(cols):
    df['Ticket Médio Mídia'] = df['Receita Mídia']/df['Compras Mídia'] if not total_flag else df['Receita Mídia']/df.loc['Total', 'Compras Mídia']

  if set(['Receita', 'Investimento']) <= set(cols):
    df['ROAS'] = df['Receita']/df['Investimento'] if not total_flag else df['Receita']/df.loc['Total', 'Investimento']

  if set(['Receita Total', 'Investimento']) <= set(cols):
    df['ROAS Total'] = df['Receita Total']/df['Investimento'] if not total_flag else df['Receita Total']/df.loc['Total', 'Investimento']

  if set(['Receita Mídia', 'Investimento']) <= set(cols):
    df['ROAS Mídia'] = df['Receita Mídia']/df['Investimento'] if not total_flag else df['Receita Mídia']/df.loc['Total', 'Investimento']

  if current_year and periods_dict:
    df = calculate_periods_variation(df, current_year, **periods_dict)

  return df

'''
Essa função recebe um dataframe como parâmetro e ordena as colunas de acordo com a configuração acima.
Colunas que não estejam na configuração são removidas.
'''
def sort_df_columns(df):
  final_order = []
  df_cols = df.columns.tolist()

  for c in focus_columns:
    if c in df_cols:
      final_order.append(c)

  return final_order

'''
Essa função recebe o editor do report, a base de dados original, o nome da plataforma que estamos trabalhando e o
dicionário com a configuração das datas que gostaríamos de analisar.
Após o processamento, uma aba será criada/atualizada no report com os dados processados.
'''
def create_report(report_sh, db, plataform_name, **kwargs):
  dates_dict = get_dates_dict(**kwargs)
  periods_dict = determine_periods(**kwargs)

  last_db_entry_date = max(db['Data'])
  current_year = f"{date_to_string(dates_dict['init_current_year'])} a {date_to_string(last_db_entry_date)}"

  remove_columns = ['Data', 'Conta', 'Campanha']
  if 'Anúncio' in db.columns.tolist():
    remove_columns = ['Data', 'Conta', 'Campanha', 'Anúncio']

  df = pd.DataFrame()
  df[current_year] = db[(db['Data'] >= date_to_string(dates_dict['init_current_year'])) & (db['Data'] <= last_db_entry_date)].loc[:, db.columns.difference(remove_columns)].sum().to_frame()
  df[periods_dict.get('current_period')] = db[(db['Data'] >= dates_dict['init_date']) & (db['Data'] <= dates_dict['end_date'])].loc[:, db.columns.difference(remove_columns)].sum().to_frame()
  df[periods_dict.get('wow_period')] = db[(db['Data'] >= dates_dict['wow_init_date']) & (db['Data'] <= dates_dict['wow_end_date'])].loc[:, db.columns.difference(remove_columns)].sum().to_frame()
  df[periods_dict.get('mom_period')] = db[(db['Data'] >= dates_dict['mom_init_date']) & (db['Data'] <= dates_dict['mom_end_date'])].loc[:, db.columns.difference(remove_columns)].sum().to_frame()
  df[periods_dict.get('yoy_period')] = db[(db['Data'] >= dates_dict['yoy_init_date']) & (db['Data'] <= dates_dict['yoy_end_date'])].loc[:, db.columns.difference(remove_columns)].sum().to_frame()

  if periods_dict.get('custom_period'):
    df[periods_dict.get('custom_period')] = db[(db['Data'] >= dates_dict['custom_init_date']) & (db['Data'] <= dates_dict['custom_end_date'])].loc[:, db.columns.difference(remove_columns)].sum().to_frame()

  if not drop_zero_cols(df).empty:
    df = df.T
    df = drop_zero_cols(df)

    df = calculate_metrics(df, plataform_name, current_year=current_year, **periods_dict)
    df.replace([np.inf, -np.inf], 0, inplace=True)

    df['Período'] = df.index
    cols = df.columns.tolist()
    cols = [cols[-1]] + cols[:-1]
    df = df[cols]

    df = df[sort_df_columns(df)]
    df = df.fillna(0)

  update_reports_sheet(report_sh, plataform_name, [df.columns.values.tolist()] + df.values.tolist())
  if plataform_name != 'Geral':
    create_periodic_report(report_sh, db, f'Controle Semanal {plataform_name}', '1W', campaign_flag=False, **kwargs)
    create_periodic_report(report_sh, db, f'Controle Mensal {plataform_name}', '1M', campaign_flag=False, **kwargs)

'''
Essa função recebe o editor do report, a base de dados mesclada com as plataformas e o Analytics CPC,
o nome da plataforma que estamos trabalhando, uma flag para agruparmos o dataframe por campanha e o
dicionário com a configuração das datas que gostaríamos de analisar.
Após o processamento, uma aba será criada/atualizada no report com os dados processados.
'''
def create_periodic_report(report_sh, general_df, plataform_name, frequency, campaign_flag, **kwargs):
  dates_dict = get_dates_dict(**kwargs)
  df = general_df.copy()

  grouper_list = [pd.Grouper(key='Data', freq=frequency)] if not campaign_flag else [pd.Grouper(key='Data', freq=frequency), pd.Grouper(key='Campanha')]
  # df = df[df['Data'] >= date_to_string(dates_dict['init_current_year'])].groupby(grouper_list, as_index=False).sum()
  df = df.groupby(grouper_list, as_index=False).sum()
  df = calculate_metrics(df, plataform_name)

  df['Data'] = df['Data'].apply(lambda row: date_to_string(row))

  df = df[sort_df_columns(df)]

  df.replace([np.inf, -np.inf], 0, inplace=True)
  df = df.fillna(0)

  update_reports_sheet(report_sh, plataform_name, [df.columns.values.tolist()] + df.values.tolist())

'''
Essa função recebe o editor do report, a base de dados de todas as plataformas, a base do Analytics e a base do Analytics CPC,
o nome da plataforma que estamos trabalhando, uma flag para agruparmos o dataframe por campanha e o
dicionário com a configuração das datas que gostaríamos de analisar.
Após o processamento, um dataframe com uma base de dados mesclada com as plataformas e o Analytics CPC será criado e
uma aba será criada/atualizada no report com os esses dados processados.
'''
def create_general_report(report_sh, plataforms_df, analytics_df, analytics_midiaPaga_df, plataform_name, campaign_flag=False, **kwargs):
  df = plataforms_df.groupby(['Data'], as_index=False).sum()
  df = df.drop(['Compras', 'Receita'], axis=1)

  df = df.merge(analytics_df[['Data', 'Compras', 'Receita']], on=['Data'])
  df.rename(columns={'Compras':'Compras Total'}, inplace=True)
  df.rename(columns={'Receita':'Receita Total'}, inplace=True)

  df = df.merge(analytics_midiaPaga_df[['Data', 'Compras', 'Receita']], on=['Data'])
  df.rename(columns={'Compras':'Compras Mídia'}, inplace=True)
  df.rename(columns={'Receita':'Receita Mídia'}, inplace=True)

  create_periodic_report(report_sh, df, 'Controle Semanal', '1W', campaign_flag, **kwargs)
  create_periodic_report(report_sh, df, 'Controle Mensal', '1M', campaign_flag, **kwargs)

  create_report(report_sh, df, plataform_name, **kwargs)

'''
Essa função recebe o editor do report, o dicionário contendo as bases de dados de todas as plataformas e
e o dicionário com a configuração das datas que gostaríamos de analisar. Em seu processamento, as diferentes bases de dados
são concatenadas para que se possa fazer uma análise das métricas de cada plataforma, bem como a análise do conjunto total.
Também é feita uma comparação, mostrando o peso em porcentagem que cada plataforma possui em cada métrica.
Esses dados processados são salvos em uma aba chamada "Distribuição".
'''
def create_plataforms_control_report(report_sh, db_dict, **kwargs):
  dates_dict = get_dates_dict(**kwargs)

  plataforms_df = pd.DataFrame()
  data_list = []

  for db_name, df in db_dict.items():
    plataform_name = db_name.split('_', 1)[-1]
    if not 'analytics' in plataform_name:
      df['Plataforma'] = plataform_name
      plataforms_df = pd.concat([plataforms_df, df]).reset_index(drop=True)

  plataforms_df = plataforms_df[(plataforms_df['Data'] >= dates_dict['init_date']) & (plataforms_df['Data'] <= dates_dict['end_date'])].drop(['Data'], axis=1).groupby(['Plataforma']).sum()
  plataforms_df.loc['Total'] = plataforms_df.sum(axis=0)

  investiment_values_df = calculate_metrics(plataforms_df.copy(), 'Distribuição')
  investiment_values_df.replace([np.inf, -np.inf], 0, inplace=True)
  investiment_values_df['Plataforma'] = investiment_values_df.index
  investiment_values_df = investiment_values_df.fillna(0)
  investiment_values_df = investiment_values_df[sort_df_columns(investiment_values_df)]

  investment_proportions_df = calculate_metrics(plataforms_df.copy(), 'Distribuição', total_flag=True)
  investment_proportions_df = investment_proportions_df.div(investment_proportions_df.iloc[-1])
  investment_proportions_df.replace([np.inf, -np.inf], 0, inplace=True)
  investment_proportions_df = investment_proportions_df.fillna(0)
  investment_proportions_df['Plataforma'] = investment_proportions_df.index
  investment_proportions_df = investment_proportions_df[sort_df_columns(investment_proportions_df)]

  data_list.append(['Valores'])
  data_list.extend([investiment_values_df.columns.values.tolist()] + investiment_values_df.values.tolist())
  data_list.append([''])
  data_list.append(['Proporções'])
  data_list.extend([investment_proportions_df.columns.values.tolist()] + investment_proportions_df.values.tolist())
  data_list.append([''])

  update_reports_sheet(report_sh, 'Distribuição', data_list)

'''
Essa função recebe o editor do report, o nome da aba a ser atualizada e a base de dados processada a ser inserida na planilha de report.
Caso a aba em questão não exista, uma função é acionada para criá-la.
Caso exista, a aba é limpa de dados antes de os novos serem inseridos na mesma.
'''
def update_reports_sheet(report_sh, sheet_name, processed_db_list):
  num_rows = len(processed_db_list)
  num_columns = max([len(p) for p in processed_db_list])
  prepare_reports_spreadsheet(report_sh, sheet_name, num_rows, num_columns)
  sheet = report_sh.worksheet(sheet_name)
  sheet.update(range_name="B2", values=processed_db_list, value_input_option="USER_ENTERED")

'''
Essa função recebe o editor do report, o nome da aba a ser atualizada o número de linhas e o número de colunas do novo report.
Caso a aba em questão não exista, uma nova é criada.
Caso exista, a aba é limpa de dados no intervalo da matriz do novo report.
'''
def prepare_reports_spreadsheet(report_sh, sheet_name, num_rows, num_columns):
  ws_list = [str(db).split("'")[1] for db in report_sh.worksheets()]

  if not sheet_name in ws_list:
    report_sh.add_worksheet(sheet_name, rows=100, cols=20)
  else:
    report_sh.values_clear(f"'{sheet_name}'!B2:{get_column_letter(num_columns)}{num_rows}")

'''
Essa função recebe o dataframe de uma das bases e faz a "tradução" dos parâmetros para que haja uniformidade entre as bases.
'''
def translate_cols(df, translation_df):
  for original_col in df.columns.tolist():

    for translated_col in translation_df.columns.tolist():

      if original_col in translation_df[translated_col].values.tolist():

        if translated_col in df.columns.tolist():
          df[translated_col] += df[original_col]
          df = df.drop([original_col], axis=1)

        else:
          df.rename(columns={original_col:translated_col}, inplace=True)

  return df

'''
Essa função recebe o editor da base de dados e o nome da aba a ser lida.
Os valores são tratados para que não haja confusões entre números e frases e um dataframe é retornado.
'''
def preprocess_dataframe(db_sh, sheet_name, translation_df):
  df = pd.DataFrame(db_sh.worksheet(sheet_name).get_all_values())

  if '' in df.iloc[0].tolist():
    df.columns = [f'{f_h}_{s_h}' if f_h and s_h != 'Campaign name' else s_h for f_h, s_h in zip(df.iloc[0,:], df.iloc[1,:])]
    df = df.drop([0,1]).reset_index(drop=True)
  else:
    df.columns = df.iloc[0].tolist()
    df = df.drop([0]).reset_index(drop=True)

  df = translate_cols(df, translation_df)
  df['Data'] = pd.to_datetime(df['Data'])
  df = df[df['Data'].notnull()].reset_index(drop=True)

  for col in df.columns:
    if df[col].dtype == object:
      try:
        df[col] = df[col].apply(lambda row: 0 if "Can't be calculated" in str(row) else row)
        df[col] = df[col].apply(lambda row: float(str(row).replace(',', '.')) if row else 0)
      except:
        continue

  df = df.fillna(0)
  df = df.infer_objects()
  df = df.drop_duplicates()

  # Retirar depois
  if 'Região' in df.columns:
    df = df.drop(['Região'], axis=1)

  if 'Campanha' in df.columns:
    df = df.drop(['Campanha'], axis=1)

  return df

'''
Essa função recebe a URL do documento de Configuração da ferramenta.
Essa é a função principal, reponsável pela execução de todo o processo.
'''
def main(config_url='https://docs.google.com/spreadsheets/d/1kH1LckYbl3ft76FBtuZw8uOwg-sD_5UQowvu-ZYa3Go/edit?usp=drive_link'):
  translation_df, config_df = open_config(config_url)

  for _, row in config_df.iterrows():
    try:
      initial_data = open_reports(row['DataBase URL'], row['Report URL'], row['Cliente'])

      if not initial_data:
        return

      db_sh, report_sh, db_names_list = initial_data

      kwargs = {
          'init_date': row['Data Inicial'],
          'end_date': row['Data Final'],
          'custom_init_date': row['Data Inicial Customizada'],
          'custom_end_date': row['Data Final Customizada']
      }

      create_config_sheet(report_sh, **kwargs)

      db_dict = {}
      plataforms_df = pd.DataFrame()

      for db_name in db_names_list:
        sheet_name = db_name.split('_', 1)[-1]
        if not 'criativos' in sheet_name:
          db_dict[db_name] = preprocess_dataframe(db_sh, db_name, translation_df)

          if not 'analytics' in sheet_name:
            create_report(report_sh, db_dict[db_name], sheet_name, **kwargs)
            plataforms_df = pd.concat([plataforms_df, db_dict[db_name]]).groupby(['Data']).sum().reset_index()

      create_general_report(report_sh, plataforms_df, db_dict['bd_analytics'], db_dict['bd_analytics_midiaPaga'], 'Geral', **kwargs)

      create_plataforms_control_report(report_sh, db_dict, **kwargs)

      print(f'Report "{report_sh.title}" atualizado com sucesso')
      print('-----------------------------------------------------------------')

    except:
      print(f'Erro encontrado durante o processamento da base de dados "{db_sh.title}"')
      print('Por favor, cheque a base de dados escolhida e os links colocados no documento de Configuração')
      print('-----------------------------------------------------------------')

    # return db_dict

main()